# Trabalho Prático 4

## Grupo 04 - Renato Garcia (A101987) & Bernardo Moniz (A102497)

## Problema 2
### Enunciado

Este exercício é dirigido à <u>prova de correção</u> do algoritmo estendido de Euclides  apresentado no trabalho TP3

a. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $\gcd$  é   $\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$.

b. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando **assert**.

c. Construa codificações do programa LPA através de transformadores de predicados “strongest post-condition” e  prove a correção  do programa LPA.


### Resolução